In [1]:
import sys
import os
sys.path.append(os.getcwd() + '/src')

In [4]:
from data.load import load_dataset, extract_targets

data = load_dataset()

提取特征

In [5]:
from feature import extract_user_feature, extract_features

user_features = extract_user_feature(data['train'], data['all_uid'])
features_train = extract_features(data['train'], user_features)
features_valid = extract_features(data['valid'], user_features)
features_test = extract_features(data['test'], user_features)
print('feature dim:', features_train.shape[1])

User Histories:   0%|          | 0/1044681 [00:00<?, ?it/s]

User Histories: 100%|██████████| 1044681/1044681 [00:24<00:00, 43025.51it/s]


feature dim: 34


In [6]:
import numpy as np
import pandas as pd

targets_train = extract_targets(data['train'], 'log')
targets_valid = extract_targets(data['valid'], 'linear')

线性回归 - 拟合+预测

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# model = LinearRegression()
model = RandomForestRegressor(n_estimators=20, n_jobs=-1)
model.fit(features_train, targets_train)
predicts = model.predict(features_valid)
predicts.shape

(184937, 3)

In [8]:
from metric import compute_metrics

# 计算指标时恢复对数
def exp_interactions(x: np.ndarray) -> np.ndarray:
    """先计算指数，然后取整"""
    return np.rint(np.exp(x) - 1).astype(int)
compute_metrics(exp_interactions(predicts), targets_valid)

{'score': 0.34364554286003113}

测试集结果提交

In [8]:
test_set_predicts = exp_interactions(model.predict(features_test))
results = []
for i in range(len(data['test'])):
    results.append(
        "{}\t{}\t{},{},{}\n".format(
            data['test'].loc[i, 'uid'],
            data['test'].loc[i, 'mid'],
            test_set_predicts[i, 1],
            test_set_predicts[i, 2],
            test_set_predicts[i, 0]
        )
    )
with open("submission.txt", 'w') as f:
    f.writelines(results)